In [ ]:
import numpy as np
import pandas as pd
import openai
import os
import logging
from dotenv import load_dotenv, dotenv_values
import json

# from helper.utils import *
# personally would not recommend this importing style, 
# it's recommend to use "from package import foo1, foo2", or "import package as p"
from helper.utils import configure_api
from helper.data_pipeline import gather_data, translate_data, analyse_data, embed_data


# load_dotenv()  # 
d = dotenv_values()
for k in d.keys():
    os.environ[k] = d[k]

# General modules

# Setup API keys
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key
client = openai.Client()

# Specify models
chat_model_name = 'gpt-4o-mini'
openai_embedding_model = "text-embedding-3-small"
local_embedding_model = "all-MiniLM-L6-v2"

configure_api(client, chat_model_name)

# Specify paths for storing (backup) data
root_dir = r'S:\SID\Analytics\Working Files\Individual\Florian\Projects\DataScience\cluster_analysis\Data\Backup'

# Setup the logger
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

logging.getLogger("httpx").setLevel(logging.ERROR)      # Supress API HTTP request logs

# path_db_prepared = os.path.join(root_dir, data_source, "db_prepared.json")          #backup
# path_db_translated = os.path.join(root_dir, data_source, "db_translated.json")      #backup
# path_db_analysed = os.path.join(root_dir, data_source, "db_analysed.json")          #backup
# path_db_embedded = os.path.join(root_dir, data_source, "db_embedded.json")          #backup


# path_db_clustered = os.path.join(root_dir, data_source, "db_clustered.json")        #backup
# path_db_final = os.path.join(root_dir, data_source, "db_final.json")                #final file

In [2]:
# Steam Reviews
from helper.redshift_conector_standalone import fetch_query_results


data_source = 'Steam'
longname = 'com.pikpok.hrc'
id_column = 'recommendationid'
text_column = 'review_text'
timestamp_column = 'timestamp_updated'
language_column = 'language'
embed_key = "sentence"  # "topic" or "sentence"


def steam_query_function():
    # SQL Query Redshift
    sql_query = """
    SELECT *
    FROM steam_review
    where app_id_name = '1166860_Rival_Stars_Horse_Racing_Desktop_Edition'
    limit 5
    """
    logger.info(f"Query Redshift with: {sql_query}")

    try:
        results_json, results_df = fetch_query_results(sql_query)
        # Print the first row of the DataFrame
        logger.info("Successfully fetched query results, with shape: %s", results_df.shape)
    except Exception as e:
        logger.error(f"Error fetching query results: {e}")
        raise
    
    return results_json


path_dir = os.path.join(root_dir, data_source)
if not os.path.exists(path_dir):
    os.mkdir(path_dir)
    
gather_data(root_dir, data_source,  
            query_function=steam_query_function, 
            id_column=id_column, 
            text_column=text_column, 
            timestamp_column=timestamp_column,
            longname=longname)
translate_data(root_dir, data_source, language_column)
analyse_data(root_dir, data_source, client, chat_model_name)
embed_data(root_dir, data_source, client, embed_key)

2025-02-18 15:43:49,502 - INFO - Query Redshift with: 
    SELECT *
    FROM steam_review
    where app_id_name = '1166860_Rival_Stars_Horse_Racing_Desktop_Edition'
    limit 5
    
2025-02-18 15:43:50,643 - INFO - Successfully fetched query results, with shape: (5, 14)
2025-02-18 15:43:50,649 - INFO - Data successfully saved to C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Steam\db_prepared.json
2025-02-18 15:43:50,656 - INFO - 'language' column already exists. Skipping language detection.
2025-02-18 15:43:50,657 - INFO - Loading existing reviews from: C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Steam\db_translated.json
2025-02-18 15:43:50,659 - INFO - Found 0 new reviews to process.
2025-02-18 15:43:50,660 - INFO - No new reviews to add. All IDs already exist.
2025-02-18 15:43:50,661 - INFO - Translation completed. Total reviews translated: 0
2025-02-18 15:43:50,666 - INFO - Skipping entry 0 (ID: 182030680) - already processed.
2025-02-18 15:43:50,667 - 

In [ ]:
# Google play review
import datetime
import google_play_scraper as gps

data_source = 'Google Play'
id_column = 'reviewId'
text_column = 'content'
timestamp_column = 'at'
language_column = 'language'
embed_key = "sentence"  # "topic" or "sentence"


def google_play_query_function():
    app_id = "com.pikpok.hrc.play"
    longname = '.'.join(app_id.split('.')[:3])
    result, continuation_token = gps.reviews(
        app_id,
#         lang="en",  # Language (English)
#         country="us",  # Country (United States)
        count=2000,  # Number of reviews to fetch
        sort=gps.Sort.NEWEST
    )
    for e in result:
        e['longname'] = longname
        for k in e.keys():
            if isinstance(e[k], datetime.datetime):
                e[k] = int(e[k].timestamp())
    return json.dumps(result)


path_dir = os.path.join(root_dir, data_source)
if not os.path.exists(path_dir):
    os.mkdir(path_dir)
    
gather_data(root_dir, data_source, 
            query_function=google_play_query_function, 
            id_column=id_column, 
            text_column=text_column, 
            timestamp_column=timestamp_column)
translate_data(root_dir, data_source, language_column)
analyse_data(root_dir, data_source, client, chat_model_name)
embed_data(root_dir, data_source, client, embed_key)

2025-02-19 09:39:11,269 - INFO - Data successfully saved to C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Google Play\db_prepared.json
2025-02-19 09:39:11,301 - INFO - Starting language detection for column: 'pp_review'


[{'reviewId': '0c1aa3b6-a4e4-4930-8a22-2a0c6d611f86', 'userName': 'Ed D', 'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjUdVVUIuJ_jQC1gpwlnbdXHDT4HNRTCUxNyQ-WqWQ42KnhPix-L', 'content': "It's a good game. However, you can not recover your extra's through commercials. So, if I can not recover anything from the commercials? Why have them?", 'score': 1, 'thumbsUpCount': 1, 'reviewCreatedVersion': '1.59.2', 'at': 1739815565, 'replyContent': None, 'repliedAt': None, 'appVersion': '1.59.2', 'longname': 'com.pikpok.hrc'}, {'reviewId': 'a45a5985-0f0a-45dd-b722-176f95c036f7', 'userName': 'Shauna Greeley', 'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjUaAYdzj_2YiCemeV3zbi2LMoeQtHeQNhAv7t6tpVgTd6G56xWU', 'content': "I paid for a one month subscription during a half-price event only to have it canceled and then refunded after only a few days, very disappointed and needless to say I don't plan on spending anymore money on this game if this is what happens.", 'score': 1, 

2025-02-19 09:39:11,529 - INFO - Language detection completed. Added column 'language'.
2025-02-19 09:39:11,529 - INFO - Loading existing reviews from: C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Google Play\db_translated.json
2025-02-19 09:39:11,539 - INFO - Found 1995 new reviews to process.
2025-02-19 09:39:11,544 - INFO - Translating review ID: fd629e60-9237-479b-8bca-56336277d5bb (Detected Language: german)
2025-02-19 09:39:12,647 - INFO - Translating review ID: 65d95435-5870-456b-8511-cc744965b0ce (Detected Language: spanish)
2025-02-19 09:39:13,061 - INFO - Translating review ID: bb5c8e73-c480-4a59-9867-d44ea56ed80b (Detected Language: spanish)
2025-02-19 09:39:13,622 - INFO - Translating review ID: 315925e5-c764-48a9-972f-2e7c564593c7 (Detected Language: german)
2025-02-19 09:39:14,079 - INFO - Translating review ID: b825066b-febf-4f78-9917-f98b1ee90944 (Detected Language: unknown)
2025-02-19 09:39:14,953 - INFO - Translating review ID: 5f7a13f8-9034-4582-aa3b-d

2025-02-19 09:40:06,923 - INFO - Translating review ID: f4f636f7-f1b6-40d6-afbf-3e19c7e08742 (Detected Language: french)
2025-02-19 09:40:07,952 - INFO - Translating review ID: 7eeb653b-0a83-4598-9760-f863d2ca3054 (Detected Language: unknown)
2025-02-19 09:40:08,457 - INFO - Translating review ID: 5f65d911-7a47-430a-86ac-0e5cff2fb96e (Detected Language: german)
2025-02-19 09:40:08,902 - INFO - Translating review ID: 861f4d63-982b-4df9-8b2d-fb17946d771e (Detected Language: french)
2025-02-19 09:40:09,350 - INFO - Translating review ID: b4ce2b31-ceb3-412c-9742-ea0c58c0dae6 (Detected Language: spanish)
2025-02-19 09:40:09,901 - INFO - Translating review ID: c76fcafc-e685-4cd4-8c2f-148bff6c6c6a (Detected Language: unknown)
2025-02-19 09:40:11,136 - INFO - Translating review ID: 824c8c71-2350-4a9c-9f03-51a3616a6e73 (Detected Language: french)
2025-02-19 09:40:11,608 - INFO - Translating review ID: 28e0a916-6320-4e6e-a4b3-ce21aaf1f3f0 (Detected Language: german)
2025-02-19 09:40:12,814 - INF

2025-02-19 09:41:26,221 - INFO - Extracting topics for entry ID 90fd5518-ea3c-4b51-9700-917df9ddf024
2025-02-19 09:41:27,227 - INFO - Analyzing sentiment for topic 'Game Enjoyment' (Entry ID 90fd5518-ea3c-4b51-9700-917df9ddf024)
2025-02-19 09:41:27,827 - INFO - Tokens used so far: Prompt Tokens: 15102, Completion Tokens: 1464
2025-02-19 09:41:27,830 - INFO - Extracting topics for entry ID 471ede79-9566-408b-a380-2e277ebbfcf0
2025-02-19 09:41:28,454 - INFO - Data successfully saved to C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Google Play\db_analysed.json
2025-02-19 09:41:28,455 - INFO - Progress saved at index 10.
2025-02-19 09:41:28,455 - INFO - Tokens used so far: Prompt Tokens: 15705, Completion Tokens: 1472
2025-02-19 09:41:28,456 - INFO - Extracting topics for entry ID 28b21ebe-cee6-42b4-b40e-e9e90232f566
2025-02-19 09:41:29,545 - INFO - Analyzing sentiment for topic 'Connectivity Issues' (Entry ID 28b21ebe-cee6-42b4-b40e-e9e90232f566)
2025-02-19 09:41:30,007 - IN

...

2025-02-19 10:55:40,042 - INFO - Tokens used so far: Prompt Tokens: 1405906, Completion Tokens: 118381
2025-02-19 10:55:40,043 - INFO - Extracting topics for entry ID 7646d505-2bb7-44c7-9510-1dcb148808ae
2025-02-19 10:55:41,148 - INFO - Analyzing sentiment for topic 'Gameplay Enjoyment' (Entry ID 7646d505-2bb7-44c7-9510-1dcb148808ae)
2025-02-19 10:55:41,665 - INFO - Tokens used so far: Prompt Tokens: 1406683, Completion Tokens: 118437
2025-02-19 10:55:41,667 - INFO - Extracting topics for entry ID bfc24564-b1ec-4bc2-8db3-a4d7e330f14e
2025-02-19 10:55:42,621 - INFO - Analyzing sentiment for topic 'General Impression' (Entry ID bfc24564-b1ec-4bc2-8db3-a4d7e330f14e)
2025-02-19 10:55:43,096 - INFO - Tokens used so far: Prompt Tokens: 1407432, Completion Tokens: 118478
2025-02-19 10:55:43,096 - INFO - Extracting topics for entry ID b29b68e1-7e88-4268-81ed-afbc77784fbd
2025-02-19 10:55:45,458 - INFO - Analyzing sentiment for topic 'Graphics Quality' (Entry ID b29b68e1-7e88-4268-81ed-afbc7778

2025-02-19 10:56:29,141 - INFO - Analyzing sentiment for topic 'Marwari Horses' (Entry ID cc0af933-a853-411a-aaea-51159667409b)
2025-02-19 10:56:29,621 - INFO - Data successfully saved to C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Google Play\db_analysed.json
2025-02-19 10:56:29,622 - INFO - Progress saved at index 1630.
2025-02-19 10:56:29,622 - INFO - Tokens used so far: Prompt Tokens: 1422487, Completion Tokens: 119973
2025-02-19 10:56:29,624 - INFO - Extracting topics for entry ID 5b5dc863-6032-41f3-85ef-6a0bb7afd9b6
2025-02-19 10:56:32,476 - INFO - Analyzing sentiment for topic 'Gold Usage for Stats' (Entry ID 5b5dc863-6032-41f3-85ef-6a0bb7afd9b6)
2025-02-19 10:56:32,869 - INFO - Analyzing sentiment for topic 'Breeding Costs' (Entry ID 5b5dc863-6032-41f3-85ef-6a0bb7afd9b6)
2025-02-19 10:56:33,301 - INFO - Analyzing sentiment for topic 'Pricing Issues' (Entry ID 5b5dc863-6032-41f3-85ef-6a0bb7afd9b6)
2025-02-19 10:56:33,699 - INFO - Tokens used so far: Prompt Tokens

2025-02-19 10:57:14,779 - INFO - Extracting topics for entry ID 8e0beb56-d615-4241-bae5-deb2fccb9a5b
2025-02-19 10:57:18,074 - INFO - Analyzing sentiment for topic 'Team Interaction' (Entry ID 8e0beb56-d615-4241-bae5-deb2fccb9a5b)
2025-02-19 10:57:18,648 - INFO - Analyzing sentiment for topic 'Competition Enhancements' (Entry ID 8e0beb56-d615-4241-bae5-deb2fccb9a5b)
2025-02-19 10:57:19,017 - INFO - Analyzing sentiment for topic 'Game Enjoyment' (Entry ID 8e0beb56-d615-4241-bae5-deb2fccb9a5b)
2025-02-19 10:57:19,707 - INFO - Analyzing sentiment for topic 'Team Identity' (Entry ID 8e0beb56-d615-4241-bae5-deb2fccb9a5b)
2025-02-19 10:57:20,278 - INFO - Data successfully saved to C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Google Play\db_analysed.json
2025-02-19 10:57:20,278 - INFO - Progress saved at index 1650.
2025-02-19 10:57:20,279 - INFO - Tokens used so far: Prompt Tokens: 1439981, Completion Tokens: 121466
2025-02-19 10:57:20,280 - INFO - Extracting topics for entry 

2025-02-19 10:57:53,296 - INFO - Progress saved at index 1670.
2025-02-19 10:57:53,297 - INFO - Tokens used so far: Prompt Tokens: 1455786, Completion Tokens: 122528
2025-02-19 10:57:53,297 - INFO - Extracting topics for entry ID e9c29891-09fb-480c-b4b2-5ed44438254f
2025-02-19 10:57:54,184 - INFO - Analyzing sentiment for topic 'Overall Enjoyment' (Entry ID e9c29891-09fb-480c-b4b2-5ed44438254f)
2025-02-19 10:57:54,662 - INFO - Tokens used so far: Prompt Tokens: 1456540, Completion Tokens: 122572
2025-02-19 10:57:54,664 - INFO - Extracting topics for entry ID 233acd15-a8e5-4dce-a7a2-5b0f6c6137a8
2025-02-19 10:57:55,682 - INFO - Analyzing sentiment for topic 'Game Enjoyment' (Entry ID 233acd15-a8e5-4dce-a7a2-5b0f6c6137a8)
2025-02-19 10:57:56,118 - INFO - Tokens used so far: Prompt Tokens: 1457317, Completion Tokens: 122626
2025-02-19 10:57:56,119 - INFO - Extracting topics for entry ID be8c0e84-8a3c-4fb1-850d-14aec13b8483
2025-02-19 10:57:56,720 - INFO - Tokens used so far: Prompt Tokens

2025-02-19 10:58:37,639 - INFO - Data successfully saved to C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Google Play\db_analysed.json
2025-02-19 10:58:37,639 - INFO - Progress saved at index 1690.
2025-02-19 10:58:37,640 - INFO - Tokens used so far: Prompt Tokens: 1472826, Completion Tokens: 123918
2025-02-19 10:58:37,641 - INFO - Extracting topics for entry ID 582d1705-f27c-46af-9792-9c2725a44244
2025-02-19 10:58:38,613 - INFO - Analyzing sentiment for topic 'Overall Enjoyment' (Entry ID 582d1705-f27c-46af-9792-9c2725a44244)
2025-02-19 10:58:39,047 - INFO - Tokens used so far: Prompt Tokens: 1473581, Completion Tokens: 123963
2025-02-19 10:58:39,048 - INFO - Extracting topics for entry ID 4bd5ced9-5d1d-44da-be6e-bfb7c927eee3
2025-02-19 10:58:40,063 - INFO - Analyzing sentiment for topic 'Overall Enjoyment' (Entry ID 4bd5ced9-5d1d-44da-be6e-bfb7c927eee3)
2025-02-19 10:58:40,673 - INFO - Tokens used so far: Prompt Tokens: 1474331, Completion Tokens: 124005
2025-02-19 10:5

In [6]:
# Zendesk CS tickets (temporarily using subcategory as the context)
import psycopg2


data_source = 'Zendesk'
id_column = 'ticket_id'
text_column = 'content'
timestamp_column = 'created_at'
language_column = 'language'
embed_key = "sentence"  # "topic" or "sentence"


def zendesk_query_function():
    conn = psycopg2.connect(host=os.getenv("POSTGRESQL_HOST"), 
                            database=os.getenv("POSTGRESQL_DATABASE"),
                            user=os.getenv("POSTGRESQL_USER"), 
                            password=os.getenv("POSTGRESQL_PASSWORD"), 
                            port=os.getenv("POSTGRESQL_PORT"))
    curr = conn.cursor()  # TODO: please fill in the query and replace the date filter
    s_query = """
    select coalesce(intent_subcategory2, coalesce(intent_subcategory1, intent_primary)) as content,
           * 
    from zendesk 
    where product = 'HRC' 
    and coalesce(intent_subcategory2, coalesce(intent_subcategory1, intent_primary)) is not null
    order by created_at desc
    limit 5
    """
    s_query = s_query.replace('>>', '>').replace('<<', '<')
    curr.execute(s_query)
    df = pd.DataFrame(curr.fetchall(), columns=[i[0] for i in curr.description])
    df['created_at'] = df['created_at'].apply(lambda x: int(x.timestamp()))
    df['longname'] = ['com.pikpok.' + str(x).lower() if x is not None else None for x in df['product']]
    return df.to_json(orient='records')


path_dir = os.path.join(root_dir, data_source)
if not os.path.exists(path_dir):
    os.mkdir(path_dir)
    
gather_data(root_dir, data_source, 
            query_function=zendesk_query_function, 
            id_column=id_column, 
            text_column=text_column, 
            timestamp_column=timestamp_column)
translate_data(root_dir, data_source, language_column)
analyse_data(root_dir, data_source, client, chat_model_name)
embed_data(root_dir, data_source, client, embed_key)

2025-02-18 15:45:50,314 - INFO - Data successfully saved to C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Zendesk\db_prepared.json
2025-02-18 15:45:50,326 - INFO - Starting language detection for column: 'pp_review'
2025-02-18 15:45:50,328 - INFO - Language detection completed. Added column 'language'.
2025-02-18 15:45:50,328 - INFO - No existing file found. Starting fresh.
2025-02-18 15:45:50,331 - INFO - Found 5 new reviews to process.
2025-02-18 15:45:50,336 - INFO - Updated file saved to: C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Zendesk\db_translated.json
2025-02-18 15:45:50,336 - INFO - Translation completed. Total reviews translated: 0
2025-02-18 15:45:50,353 - INFO - Tokens used so far: Prompt Tokens: 4432, Completion Tokens: 384
2025-02-18 15:45:50,354 - INFO - Extracting topics for entry ID 101047
2025-02-18 15:45:51,303 - INFO - Analyzing sentiment for topic 'Store Accessibility' (Entry ID 101047)
2025-02-18 15:45:51,983 - INFO - Tokens used s

In [15]:
# Surveys
data_source = 'Survey'
file = r'C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\2024 Trimester 1.xlsx'
longname = 'com.pikpok.hrc'
id_column = 'review_id'
text_column = "24_[OPTIONAL] Is there anything you'd like to share about the game (good, bad, frustrating, improvement, wishlist, etc.)?"
timestamp_column = 'End Date'
language_column = 'language'
embed_key = "sentence"  # "topic" or "sentence"


def survey_query_function(file):
    file_name, file_extension = os.path.splitext(file)
    if '.csv' == file_extension:
        df = pd.read_csv(file)
        js = json.loads(df.to_json(orient='records'))
    elif '.xls' == file_extension or '.xlsx' == file_extension:
        df = pd.read_excel(file)
        js = json.loads(df.to_json(orient='records'))
    elif '.txt' == file_extension:
        with open(file, 'r') as f:
            js = json.load(f)
    for e in js:
        if 'Respondent ID' in e.keys() and 'pcubed_id' in e.keys() and 'review_id' not in e.keys():
            e['review_id'] = str(e['Respondent ID']) + ':' + str(e['pcubed_id'])
    return json.dumps(js[:5])


path_dir = os.path.join(root_dir, data_source)
if not os.path.exists(path_dir):
    os.mkdir(path_dir)
    
gather_data(root_dir, data_source, 
            query_function=survey_query_function,
            query_function_args=[file,],
            id_column=id_column, 
            text_column=text_column, 
            timestamp_column=timestamp_column)
translate_data(root_dir, data_source, language_column)
analyse_data(root_dir, data_source, client, chat_model_name)
embed_data(root_dir, data_source, client, embed_key)

2025-02-18 15:51:18,409 - INFO - Data successfully saved to C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Survey\db_prepared.json
2025-02-18 15:51:18,424 - INFO - Starting language detection for column: 'pp_review'
2025-02-18 15:51:18,427 - INFO - Language detection completed. Added column 'language'.
2025-02-18 15:51:18,427 - INFO - No existing file found. Starting fresh.
2025-02-18 15:51:18,434 - INFO - Found 5 new reviews to process.
2025-02-18 15:51:18,442 - INFO - Updated file saved to: C:\Users\mshen\Documents\Msheng_Domestic\sentiment_analysis\Survey\db_translated.json
2025-02-18 15:51:18,442 - INFO - Translation completed. Total reviews translated: 0
2025-02-18 15:51:18,457 - INFO - Tokens used so far: Prompt Tokens: 8187, Completion Tokens: 594
2025-02-18 15:51:18,458 - INFO - Extracting topics for entry ID 114585000000:531632623
2025-02-18 15:51:23,968 - INFO - Analyzing sentiment for topic 'Pasture Usability' (Entry ID 114585000000:531632623)
2025-02-18 15:51:2